In [1]:
from bs4 import BeautifulSoup
import requests
import lxml
from selenium import webdriver
import time
import numpy as np
import pandas as pd

Collect all ads of a given page

In [ ]:
driver = webdriver.Chrome()
url = 'https://www.immoweb.be/en/search/apartment/for-rent?countries=BE&page=1&orderBy=newest'
driver.get(url)
time.sleep(3)
try:
    assert '<h1 class="title--2--dark page-error__title"> Unfortunately, something went wrong.</h1>' not in driver.page_source
    soup = BeautifulSoup(driver.page_source, 'lxml')
    set_urls = set()
    for image in soup.find_all("a", attrs={"class": "card__title-link"}):
        set_urls.add(image.get("href"))
except:
    driver.close()

Scrape a given ad

In [ ]:
# scrape an ad
url = 'https://www.immoweb.be/en/classified/apartment/for-rent/hamme/9220/9995786?searchId=62be9f60a9c7a'
driver = webdriver.Chrome()
driver.get(url)
data = pd.read_html(driver.page_source)
driver.close()

# add the ad to a dictionary
ads = dict()
ads[url] = data
print(ads['https://www.immoweb.be/en/classified/apartment/for-rent/hamme/9220/9995786?searchId=62be9f60a9c7a']) # list of DataFrames

# concatenate the DataFrames in the list into a single one
df = pd.DataFrame()
for i in range(len(data)):
    df = pd.concat([df, data[i]], axis=0).reset_index(drop=True)
print(df)

Scrape an ad from its url

In [2]:
def scrape_ad(driver, url):
    driver.get(url)
    return pd.read_html(driver.page_source)

Create a dictionary of ads from a given page

In [3]:
driver = webdriver.Chrome()
ads = dict()
n = 1
while True:
    driver.get(f'https://www.immoweb.be/en/search/apartment/for-rent?countries=BE&page={n}&orderBy=newest') # request page n
    time.sleep(2) # wait for page to load properly
    soup = BeautifulSoup(driver.page_source, 'lxml') # parse page
    for ad_link in soup.find_all("a", attrs={"class": "card__title-link"}): # parse every ad link in page
        url = ad_link.get("href") # assign link to "url"
        if url not in ads:
            # print(url)
            try:
                data = scrape_ad(driver, url)
            except:
                time.sleep(2)
                try:
                    data = scrape_ad(driver, url)
                except:
                    print(f'Failed to scrape this ad: {url}') 
        df = pd.DataFrame()
        for i in range(len(data)):
            df = pd.concat([df, data[i]], axis=0).reset_index(drop=True)
            ads[url] = df

    # for testing on the first page only
    n += 1
    if n == 2:
        break

driver.close()
len(ads)

38

In [4]:
ads # dictionary of DataFrames

{'https://www.immoweb.be/en/classified/apartment/for-rent/wevelgem/8560/9996107?searchId=62bec75873da6':                                                     0  \
 0                                      Available date   
 1                                  Building condition   
 2                                 Number of frontages   
 3                              Covered parking spaces   
 4                                        Kitchen type   
 5                                            Bedrooms   
 6                                           Furnished   
 7                                             Terrace   
 8                          Primary energy consumption   
 9                                        Energy class   
 10                                                NaN   
 11                 Reference number of the EPC report   
 12                                       CO₂ emission   
 13        Yearly theoretical total energy consumption   
 14                       

In [7]:
ads['https://www.immoweb.be/en/classified/apartment/for-rent/bruxelles/1000/9996090?searchId=62bec75873da6']

,0,1
0,Available date,June 25 2022 - 12:00 AM
1,Neighbourhood or locality,Bruxelles
2,Construction year,1900
3,Number of floors,3
4,Building condition,As new
5,Street frontage width,10 m
6,Number of frontages,2
7,Surroundings type,Fitted out
8,Living area,100 m² square meters
9,Living room surface,36 m² square meters
